In [4]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session





In [6]:
bucket_name = "trybucketn2"  #Change according to my bucket name
my_region = boto3.session.Session().region_name
print(my_region)





us-east-2


In [8]:

# s3var = boto3.resource('s3')
# try:
#     if my_region == 'us-east-2' :
#         s3var.create-bucket(Bucket = bucket_name)        
#     print("S3 bucket create successfully")
# except Exception:
#     print("Error",Exception)
#created manually







In [9]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://trybucketn2/xgboost-as-a-built-in-algo/output


In [ ]:
#save model over this path

In [10]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data error :',e)
    
    

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)




Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [12]:
#train test split

import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(test_data.shape)

print(train_data.shape)






(12357, 61)
(28831, 61)


In [13]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
#save file\



boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

#upload to sagemaker s3 bucket
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [14]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

#sagemaker.TrainingInput


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


<h1> Inbuilt Algo Xgboost</h1>

In [15]:
from sagemaker.amazon.amazon_estimator import get_image_uri
# container = get_image_uri(,region_name, 'xgboost')

container = get_image_uri(boto3.Session().region_name, 'xgboost')



'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
There is a more up to date SageMaker XGBoost image. To use the newer image, please set 'repo_version'='1.0-1'. For example:
	get_image_uri(region, 'xgboost', '1.0-1').
